參考文獻
1. 撰寫第一支CNN 程式 -- 比較 『阿拉伯數字』辨識力(https://ithelp.ithome.com.tw/articles/10191924)
2. Keras 模型、函數及參數使用說明(https://dotblogs.com.tw/greengem/2017/12/17/094023)
3. Keras函数式模型接口(https://keras-cn.readthedocs.io/en/latest/models/model/)

In [1]:
# 匯入我們要用的模組
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop, Adam
import os

Using TensorFlow backend.


In [2]:
batch_size = 128 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 10 # 訓練的 epochs 數量

# 讀取資料並檢視
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# 對 label 進行 one-hot encoding (y_trian 原本是純數字)
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


x_train shape: (50000, 32, 32, 3)
代表著有 50000個樣本，每個樣本分別是 32*32像素，且有R、G、B 3個色版。

## 首先我們使用一般的 DNN (MLP) 來訓練
由於 DNN 只能輸入一維的資料，我們要先將影像進行攤平，若 (50000, 32, 32, 3) 的影像，攤平後會變成 (50000, 32x32x3) = (50000, 3072)

In [3]:
# 將資料攤平成一維資料
x_train = x_train.reshape(50000, 3072) 
x_test = x_test.reshape(10000, 3072)

# 將資料變為 float32 並標準化
# 轉換色彩 0~255 資料為 0~1
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 3072)
50000 train samples
10000 test samples


x_train shape: (50000, 3072)
代表著有 50000個樣本，每個像素及色板的排列組合共有 32x32x3 = 3072種。

In [4]:
# 建立模型(Model)：首先，我們要確立Input格式、要經過幾層處理、每一層要作甚麼處理。
# 建立簡單的線性執行的模型
model = Sequential()

# Add Input layer, 有 512個輸出變數
model.add(Dense(512, activation='relu', input_shape=(3072,)))
# Add Dropout layer,隨機斷開輸入神經元，用於防止過度擬合，斷開比例:0.2
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
# Add output layer，共有10個輸出變數(在本案例 num_classes=10，Cifar 10 共有 10 個類別)
model.add(Dense(num_classes, activation='softmax'))
# 輸出模型概況
model.summary()

# 編譯: 選擇損失函數、優化方法及成效衡量方式
# 確立目標及求解方法：以compile函數定義損失函數(loss)、優化函數(optimizer)及成效衡量指標(mertrics)。
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

# 訓練：以compile函數進行訓練，指定訓練的樣本資料(x, y)，並撥一部分資料作驗證，還有要訓練幾個週期、訓練資料的抽樣方式。
# 訓練過程會存在 history 變數中
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

# 顯示損失函數、訓練成果(分數)
# verbose：日誌顯示，0為不在標準輸出流輸出日誌信息，1為輸出進度條記錄，2為每個epoch輸出一行記錄。
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               1573376   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 1,841,162
Trainable params: 1,841,162
Non-trainable params: 0
_________________________________________________________________


Instruction

## 接下來我們使用 CNN 來訓練神經網路
CNN 的原理非常適合處理影像類的資料，就讓我們來看看，同樣的訓練條件，CNN 是否顯著優於 DNN 呢?

In [9]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# 將資料變為 float32 並標準化
# 轉換色彩 0~255 資料為 0~1
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to binary class matrices.
# y 值轉成 one-hot encoding，轉成10類
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_train.shape[1], 'y_train shape')
print(y_test.shape[1], 'y_test shape')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
10 y_train shape
10 y_test shape


In [10]:
# 建立簡單的線性執行的模型
model = Sequential()
# 建立卷積層，filter=32,即 output space 的深度, Kernal Size: 3x3, padding='same'不足的空間用 0填充一列
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
# activation function 採用 relu
model.add(Activation('relu'))
# 建立卷積層，filter=32,即 output size, Kernal Size: 3x3
model.add(Conv2D(32, (3, 3)))
# activation function 採用 relu
model.add(Activation('relu'))
# 建立池化層，池化大小=2x2，取最大值
model.add(MaxPooling2D(pool_size=(2, 2)))
# 隨機斷開輸入神經元，用於防止過度擬合，斷開比例:0.25
model.add(Dropout(0.25))

# 建立卷積層，filter=64,即 output size, Kernal Size: 3x3, padding='same'不足的空間用 0填充一列
model.add(Conv2D(64, (3, 3), padding='same'))
# activation function 採用 relu
model.add(Activation('relu'))
# 建立卷積層，filter=64,即 output size, Kernal Size: 3x3
model.add(Conv2D(64, (3, 3)))
# activation function 採用 relu
model.add(Activation('relu'))
# 建立池化層，池化大小=2x2，取最大值
model.add(MaxPooling2D(pool_size=(2, 2)))
# 隨機斷開輸入神經元，用於防止過度擬合，斷開比例:0.25
model.add(Dropout(0.25))

# 把多維的輸入一維化，常用在從卷積層到全連接層的過渡。
model.add(Flatten())
# 全連接層: 512個output
model.add(Dense(512))
# activation function 採用 relu
model.add(Activation('relu'))
# 隨機斷開輸入神經元，用於防止過度擬合，斷開比例:0.5
model.add(Dropout(0.5))
# 使用 softmax activation function，並將結果分類
model.add(Dense(num_classes))
model.add(Activation('softmax'))
# 輸出模型概況
model.summary()

# 編譯: 選擇損失函數、優化方法及成效衡量方式
# 確立目標及求解方法：以compile函數定義損失函數(loss)、優化函數(optimizer)及成效衡量指標(mertrics)。
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

# 訓練：以compile函數進行訓練，指定訓練的樣本資料(x, y)，並撥一部分資料作驗證，還有要訓練幾個週期、訓練資料的抽樣方式。
# 訓練過程會存在 history 變數中
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

# 顯示損失函數、訓練成果(分數)
# verbose：日誌顯示，0為不在標準輸出流輸出日誌信息，1為輸出進度條記錄，2為每個epoch輸出一行記錄。
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 64)        18496     
_________

In [13]:
# 模型結構存檔
from keras.models import model_from_json
json_string = model.to_json()
with open("cnn.config", "w") as text_file:
    text_file.write(json_string)

# 模型訓練結果存檔
model.save_weights("cnn.weight")

## 同樣運算 10 個 epochs，但 CNN 在 test data 的準確率顯著優於 DNN!

## 作業
1. 請試著調整各個超參數，並說明那些超參數對於結果有明顯的影響?
2. CNN 與 DNN 哪個模型的參數數量比較多? 造成參數的數量不同的原因在哪?

1. Conv2D 層數 & 優化器有較明顯的影響。
2. DNN參數量:1,841,162，CNN參數量:1,250,858，DNN參數量較多。CNN每一層input output的數值比較小。